In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import sys

#Crawl for Cuisines
home_link='http://www.bbc.co.uk'
start_link="http://www.bbc.co.uk/food/recipes"
#Extracting the data from the webpages

def extract_method(rsoup):
    z_tmp=rsoup.find_all('p',{'class':'recipe-method__list-item-text'})
    txt=[]
    for i in z_tmp:
        txt.append(i)
        
    method_info=''
    for tag in txt:
        b=BeautifulSoup(str(tag))
        method_info=method_info+(b.find('p').contents[0])
        
    return ' '.join(method_info.split())


def extract_author(rsoup):
    result = ''
    for tag in rsoup.find_all(itemprop='author'):
        result = tag.contents[0]
    return result


def extract_cooking_time(rsoup):
    result = ''
    for tag in rsoup.find_all(itemprop='cookTime'):
        result = tag.contents[0]
    return result


def extract_dietary_information(rsoup):
    s="Vegetarian"
    items = rsoup.find_all("div", {"class":"recipe-metadata__dietary"})
    if bool(items)==True:
        for i in items:
            x = i.find("p", {"class": "recipe-metadata__dietary-vegetarian-text"}).get_text()
            if s in x:
                return(s)      
        
    else:
        return("No information")
    
def extract_ingredients(rsoup):
    
    items = rsoup.find_all("ul", {"class":"recipe-ingredients__list"})
    ingredients_info=''
    if bool(items)==True:
        for i in items:
            x = i.find_all("li", {"class": "recipe-ingredients__list-item"})
            for j in x:
                ingredients_info+=(j.get_text())
                
    else:
        return ('No Information')

    return ' '.join(ingredients_info.split())

    
def extract_prep_time(rsoup):
    result = ''
    for tag in rsoup.find_all(itemprop='prepTime'):
        result = tag.contents[0]
    return result


def extract_program_information(rsoup):
    
    items = rsoup.find_all("div", {"class":"recipe-is-from-widget__about"})
    if bool(items)==True:
        for i in items:
            x = i.find("p", {"class": "recipe-is-from-widget__programme-series-title"}).get_text()
            return(x.strip())
        
    else:
        return("No information")

def extract_serves(rsoup):
    result = ''
    for tag in rsoup.find_all(itemprop='recipeYield'):
        result = tag.contents[0]
    return result

def extract_title(rsoup):
    result = ''
    for tag in rsoup.find_all(itemprop='name'):
        result = tag.contents[0]
    return result

#File management functions

def collect_all_information(url):
    ## Use the Functions to get the Data #
    try:
        recipe=requests.get(url)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        print(e)
        sys.exit(1)
    rsoup = BeautifulSoup(recipe.text, "lxml")

    Single_Recipe_Data=[]
    auth=extract_author(rsoup)
    p_time=extract_prep_time(rsoup)
    c_time=extract_cooking_time(rsoup)
    sv=extract_serves(rsoup)
    dietary=extract_dietary_information(rsoup)
    progrm=extract_program_information(rsoup)
    ing=extract_ingredients(rsoup)
    m=extract_method(rsoup)
    t=extract_title(rsoup)
    # Save the Data together in one, Variable , and return that Variable
    Single_Recipe_Data.append(t)
    Single_Recipe_Data.append(auth)
    Single_Recipe_Data.append(p_time)
    Single_Recipe_Data.append(c_time)
    Single_Recipe_Data.append(dietary)
    Single_Recipe_Data.append(progrm)
    Single_Recipe_Data.append(sv)
    Single_Recipe_Data.append(ing)
    Single_Recipe_Data.append(m)
    
    return Single_Recipe_Data

def write_to_file(Recipe,file_name):
    target=open(file_name,'a',encoding='utf-8')
    for element in Recipe:
        target.write(str(element))
        target.write('\n')
    target.close()
    

def read_all_links(file_name):

    target=open(file_name, 'r',encoding='utf-8')
    links=[wrd.strip() for wrd in target]

    target.close()
    return links


def extract_all_information(batch,c):
    #d=make_batches('all_recipe_links.txt')
    path = './ADM_dataset/test/___'
    #c=0
    #dt=0
    #for batch in d:
    #    print('Batch num'+str(dt)+'starting.....')
    for link in batch:
        print(link)
        print('\n')
        print(c)
        single_recipe=collect_all_information(link)
        write_to_file(single_recipe,path+str(c)+'.txt')
        time.sleep(1)
        c=c+1
        #print('Batch num '+str(dt)+'complete !!')
        #dt=dt+1
    return 'done'

def make_batches(file_name_links,n=1000):
    l = read_all_links(file_name_links)
    k=[l[i:i + n] for i in range(0, len(l), n)]
    return k

d=make_batches('all_recipe_links.txt')
all_links = read_all_links('all_recipe_links.txt')


In [2]:
path = './ADM_dataset/test'

In [22]:
import os
lst=[]
for i in range(11269):
    statinfo = os.stat(path+'/___'+str(i)+'.txt')
    if statinfo.st_size < 100:
        lst.append(i)

In [23]:
len(lst)


310